<a href="https://colab.research.google.com/github/rath772k/Machine-learning-problems/blob/master/dog_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Mounting drive

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rath772k","key":"10f609c84776cce235080e8155149c07"}'}

##Downloading Dataset

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dog-breed-identification

  0% 0.00/214k [00:00<?, ?B/s]
100% 214k/214k [00:00<00:00, 63.3MB/s]
  0% 0.00/281k [00:00<?, ?B/s]
100% 281k/281k [00:00<00:00, 88.2MB/s]
 95% 330M/346M [00:02<00:00, 82.8MB/s]
100% 346M/346M [00:02<00:00, 135MB/s] 
 97% 334M/345M [00:02<00:00, 159MB/s]
100% 345M/345M [00:02<00:00, 138MB/s]


In [4]:
!7z x "train.zip"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 361279070 bytes (345 MiB)

Extracting archive: train.zip
 40% 4096 Open              --
Path = train.zip
Type = zip
Physical Size = 361279070

  0%      3% 418 - train/0a0c223352985ec154fd604d7ddceabd.jpg                                                       7% 758 - train/12c9f56e73ac9f4a08afb142f89051ae.jpg                                                       9% 1037 - train/19a8c1698819d382c3ead14bdc1a360b.jpg      

In [0]:
!rm "train.zip"

In [6]:
!7z x "test.zip"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 362738853 bytes (346 MiB)

Extracting archive: test.zip
 39% 4096 Open              --
Path = test.zip
Type = zip
Physical Size = 362738853

  0%      3% 418 - test/09db0bb220cdbbc0adca63e38e64ab38.jpg                                                      6% 750 - test/11754ee6d33fceb7a8dab2c33f3ba008.jpg                                                      9% 1026 - test/186ce2324e1dc6ae0f0fe51811a9c2a3.jpg                  

In [0]:
!rm "test.zip"

In [8]:
!7z x "labels.csv.zip"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 218954 bytes (214 KiB)

Extracting archive: labels.csv.zip
--
Path = labels.csv.zip
Type = zip
Physical Size = 218954

  0%    Everything is Ok

Size:       482063
Compressed: 218954


In [0]:
!rm "labels.csv.zip"

In [10]:
!7z x "sample_submission.csv.zip"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 288160 bytes (282 KiB)

Extracting archive: sample_submission.csv.zip
--
Path = sample_submission.csv.zip
Type = zip
Physical Size = 288160

  0%    Everything is Ok

Size:       25200295
Compressed: 288160


In [0]:
!rm "sample_submission.csv.zip"

##Importing libraries

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from skimage import io,transform
from torchvision import models, transforms, utils
import cv2
import time
import os
import copy
from PIL import Image

##Load data

In [0]:
labels = pd.read_csv("labels.csv")

In [4]:
dat = pd.read_csv("sample_submission.csv")
dat.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333

In [5]:
labels.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07.jpg,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97.jpg,dingo
2,001cdf01b096e06d78e9e5112d419397.jpg,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d.jpg,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62.jpg,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d.jpg,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85.jpg,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b.jpg,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9.jpg,basenji
9,0042188c895a2f14ef64a918ed9c7b64.jpg,scottish_deerhound


In [20]:
labels["id"] +=".jpg"
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07.jpg,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97.jpg,dingo
2,001cdf01b096e06d78e9e5112d419397.jpg,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d.jpg,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62.jpg,golden_retriever


In [0]:
classes = pd.get_dummies(labels["breed"])

In [0]:
classes = classes.columns

In [90]:
classes

Index(['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale',
       'american_staffordshire_terrier', 'appenzeller', 'australian_terrier',
       'basenji', 'basset', 'beagle',
       ...
       'toy_poodle', 'toy_terrier', 'vizsla', 'walker_hound', 'weimaraner',
       'welsh_springer_spaniel', 'west_highland_white_terrier', 'whippet',
       'wire-haired_fox_terrier', 'yorkshire_terrier'],
      dtype='object', length=120)

In [0]:
labels.to_csv("labels.csv", index=False)

In [21]:
img_name = labels.iloc[0, 0]
print('Image name: {}'.format(img_name))

Image name: 000bec180eb18c7604dcecc8fe0dba07.jpg


##Custom dataset and dataloader

In [0]:
torch.set_default_tensor_type('torch.FloatTensor')

In [0]:
class DatasetFromCSV(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.classes = np.array(pd.get_dummies(self.labels["breed"]))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.labels.iloc[idx].loc['id'])
        image = Image.open(img_name)
        #image = image.transpose((2, 0, 1))
        classes = self.classes[idx]
        classes = np.argmax(classes)

        if self.transform:
            image = self.transform(image)
        sample = (image,classes)
        return sample

In [0]:
path = "/content/train"
csv_file = "labels.csv"
dataset = DatasetFromCSV(csv_file,path,transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]))
batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)
dataloaders = {'train':train_loader, 'val':validation_loader}

In [55]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_size = len(train_indices)
val_size = len(val_indices)
dataset_sizes = {'train' : train_size, 'val' : val_size}
print("train : "+str(train_size)+" images")
print("validation : "+str(val_size)+" images")

train : 8178 images
validation : 2044 images


##Train model function

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    #print(labels)
                    #print(outputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)
            if phase == 'train':
                scheduler.step(running_loss)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if (phase == 'val' or phase == 'test') and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

##Training

In [0]:
model = models.resnext101_32x8d(pretrained=True)

In [0]:
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs,120)

In [49]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [50]:
for name,child in model.named_children():
  print(name)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [51]:
for name, child in model.named_children():
   if name in ['fc']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False

conv1 is frozen
bn1 is frozen
relu is frozen
maxpool is frozen
layer1 is frozen
layer2 is frozen
layer3 is frozen
layer4 is frozen
avgpool is frozen
fc is unfrozen


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.004)
exp_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='min', factor=0.1, patience=5, verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

In [74]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=6)

Epoch 0/5
----------
train Loss: 0.2914 Acc: 0.9254
val Loss: 0.3027 Acc: 0.9095

Epoch 1/5
----------
train Loss: 0.2830 Acc: 0.9254
val Loss: 0.3005 Acc: 0.9110

Epoch 2/5
----------
train Loss: 0.2766 Acc: 0.9286
val Loss: 0.2941 Acc: 0.9080

Epoch 3/5
----------
train Loss: 0.2796 Acc: 0.9290
val Loss: 0.2882 Acc: 0.9105

Epoch 4/5
----------
train Loss: 0.2765 Acc: 0.9314
val Loss: 0.2868 Acc: 0.9149

Epoch 5/5
----------
train Loss: 0.2649 Acc: 0.9331
val Loss: 0.3048 Acc: 0.9061

Training complete in 59m 43s
Best val Acc: 0.914873


In [75]:
torch.save(model,"/content/drive/My Drive/Dog_resnext101")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist

##Testing

In [98]:
root_dir = '/content/test'
tsfms = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ])
model.eval()
for i in range(len(dat)):
  img_name = os.path.join(root_dir,dat['id'].iloc[i]+'.jpg')
  image = Image.open(img_name)
  image = tsfms(image)
  image = image.view(1,3,224,224)
  image = image.to(device)
  with torch.no_grad():
    op = torch.nn.functional.softmax(model(image),dim=1)
    op = op.to(torch.device('cpu'))
    op = op.numpy()
  dat.iloc[i,1:] = op.reshape(120)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [99]:
dat.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.000012,0.000008,2.112934e-07,9.713467e-08,7.605408e-08,0.000002,2.206451e-06,6.826724e-07,5.759989e-07,0.000005,9.594668e-07,1.584528e-06,4.149294e-07,0.012930,9.718711e-08,4.710568e-07,1.284605e-06,3.604474e-07,0.000004,0.000013,6.415439e-07,0.000002,0.000017,1.195476e-06,0.000003,8.306657e-08,9.441499e-08,1.208420e-07,6.735255e-08,0.000009,3.386332e-07,0.000004,0.000012,0.000005,2.928908e-07,1.007057e-07,1.662526e-07,2.021056e-08,1.105199e-07,...,2.857424e-08,2.345893e-08,0.000001,2.057892e-06,3.634119e-04,0.007811,1.848438e-07,4.143132e-06,0.000001,1.005424e-07,3.226095e-07,9.267699e-08,4.545090e-07,6.933472e-06,2.409612e-06,9.315532e-08,1.919008e-07,2.723022e-07,1.780367e-05,0.000027,0.001714,5.616126e-07,0.000004,7.743214e-07,2.588332e-07,3.292925e-07,2.191309e-07,0.000010,4.856536e-08,3.278064e-05,0.000005,1.925802e-06,1.192099e-07,0.000001,3.317182e-07,1.919561e-07,4.386271e-07,2.851086e-07,6.215338e-06,4.201142e-06
1,00102ee9d8eb90812350685311fe5890,0.000003,0.000003,1.773151e-06,1.599413e-06,8.938778e-06,0.000020,5.502108e-06,8.372335e-06,1.277789e-06,0.000001,2.046322e-06,3.372572e-05,1.313123e-06,0.000008,2.454334e-06,1.057155e-06,6.740100e-05,1.202886e-06,0.000009,0.000011,4.429394e-06,0.000002,0.000005,1.304863e-06,0.000006,1.167900e-06,9.280443e-07,1.839796e-05,5.428876e-06,0.000007,4.712027e-04,0.000003,0.000002,0.000202,1.486399e-06,5.997136e-07,1.939727e-05,4.672908e-05,9.690118e-06,...,3.066551e-05,6.503714e-06,0.000043,2.690245e-06,8.307673e-06,0.000017,2.297540e-05,1.133624e-04,0.000043,8.555772e-06,1.470722e-06,2.206255e-05,8.348823e-06,2.177740e-06,9.834004e-01,4.339816e-05,3.375678e-06,2.422864e-06,2.994161e-06,0.000017,0.000003,8.379395e-03,0.000002,5.294803e-06,8.638391e-06,3.030518e-05,1.539712e-06,0.000004,1.051438e-04,2.114568e-06,0.000008,2.109900e-07,1.753873e-06,0.000003,2.582961e-06,1.190421e-06,6.009205e-05,1.739826e-06,2.363017e-06,6.068123e-07
2,0012a730dfa437f5f3613fb75efcd4ce,0.000019,0.012046,1.336056e-04,5.813553e-05,4.508761e-05,0.000201,2.141176e-05,2.346020e-05,2.353764e-04,0.000060,2.275438e-04,2.034933e-04,1.031899e-04,0.000203,3.203584e-05,3.099566e-04,1.483172e-04,2.079622e-05,0.074681,0.000024,1.397559e-05,0.000248,0.000015,2.840568e-04,0.001193,5.025552e-05,1.717395e-05,1.364358e-05,2.653898e-05,0.000009,2.743700e-06,0.001363,0.001158,0.000435,8.415493e-05,7.072189e-05,7.119423e-05,3.419284e-05,7.206189e-05,...,5.455822e-06,1.972297e-06,0.000113,1.483505e-04,3.651287e-04,0.000040,7.725419e-06,2.946952e-05,0.000036,1.542544e-04,1.439862e-04,1.691089e-05,8.946421e-05,1.859134e-02,7.325802e-05,2.927066e-06,2.238199e-05,1.244075e-03,4.311119e-04,0.000081,0.000085,3.412944e-05,0.000049,3.644819e-04,1.545148e-05,4.521893e-04,2.785650e-04,0.000055,4.396487e-06,1.165379e-04,0.000019,7.941960e-06,9.054452e-05,0.000714,1.061792e-03,4.982524e-03,3.033064e-05,2.919509

In [0]:
dat.to_csv('/content/sample_submission.csv',index = False)